# init AD9174 for use with on board reference
This example uses the `HMC7044` and the 122.88 MHz reference oscillator on the `AD9174-FMC_EBZ` board to generate a DAC sampling clock. No external clock needed.

Requires litex_server running and connected to the VC707 USB-uart port:

```bash
litex_server --uart --uart-port /dev/ttyUSB0 --uart-baudrate 115200

```

In [1]:
import sys
from numpy import *
from time import sleep
from litex import RemoteClient
from ad9174 import Ad9174Settings, Ad9174Init

def getId(r):
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s

In [2]:
r = RemoteClient(csr_csv='../build/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'AD9174 + VC707 test 2021-02-22 11:39:46'

In [3]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)

----------------
 JESD mode 20
----------------
INTERP_CH: 1  INTERP_MAIN: 1  DSP_CLK_DIV: 16
JESD204BSettings(): 5a 05 00 87 00 1f 00 0f 2f 23 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   8        SCR:   1 
           F:   1          K:  32          M:   1          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   4 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   4 ]


In [4]:
ad = Ad9174Init(r, settings)
wr = ad.ad.wr
rr = ad.ad.rr

# 0. setup clocking
See here for the clock planning:
https://docs.google.com/spreadsheets/d/1F6s6cVM1Lo6IOUgZoq9xm0ueGYkePZFeD96N0-kPR9o/edit?usp=sharing

In [5]:
N_HMC = 20
f_VCO2 = 122.88e6 * N_HMC
ad.hmc.init_hmc7044_int(N2=N_HMC)

DIV_DAC = 2
DIV_DSP = 8
ad.hmc.setup_channel( 2, DIV_DAC)  # Clock to the AD9174
ad.hmc.setup_channel(12, DIV_DSP)  # DSP clock divider to the FPGA

# SYSREF must align with the multiframe clock and the DSP clock!
# FPGA generates 4 frames / DSP clock
# DAC expects k = 32 frames / multiframe 
# --> lmfc_cycles = 8 DSP clocks / multiframe
DIV_SYSREF = DIV_DSP * 8 * 20
ad.hmc.setup_channel( 3, DIV_SYSREF)  # SYSREF to AD9174
ad.hmc.setup_channel(13, DIV_SYSREF)  # SYSREF to FPGA
ad.hmc.trigger_div_reset()

print("Expected values:")
print(f' f_VCO: {f_VCO2 / 1e6:8.3f} MHz')
print(f'f_jesd: {f_VCO2 / DIV_DSP / 1e6:8.3f} MHz')
print(f' f_ref: {f_VCO2 / DIV_SYSREF / 1e6:8.3f} MHz')

Expected values:
 f_VCO: 2457.600 MHz
f_jesd:  307.200 MHz
 f_ref:    1.920 MHz


In [6]:
print("Measured values:")
ad.fpga_print_clocks()

Measured values:
f_jesd = 307.197606 MHz  f_ref = 1.919985 MHz


# 1. init AD9174

In [7]:
ad.init_ad9174(ADC_CLK_DIV=4, SYSREF_ERR_WINDOW=1, USE_PLL=True, M_DIV=2, N_DIV=2, OUT_DIV=2)

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DAC PLL locked: 1
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0


# 3. Init the FPGA side

In [52]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH! But sometimes needed.)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core

print('status: {:03b}'.format(r.regs.control_status.read()))

status: 000


In [53]:
# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b10)
# bit2: /jsync,  bit1: links_ready,  bit0: phys_ready
print('status: {:03b}'.format(r.regs.control_status.read()))

status: 111


In [55]:
# This number should not increase when the link is up
r.regs.control_jsync_errors.read()

236500426

if you get `status: 111` and the `jsync_errors` number does not increase, then the link is up and you're in business!!!

In [56]:
ad.trigger_jref_sync()
ad.print_irq_flags(True, True)
if ad.print_irq_flags(True):
    print('😭')
else:
    print('😃')

SYNC_ROTATION_DONE 1
DYN_LINK_LATENCY  2 cycles
😃


# 4. Test link

In [51]:
ad.print_irq_flags(True)
print()

ad.print_ilas()

ad.print_lane_status()
print()

ad.test_stpl()


JESD settings, received on lane 0 vs (programmed):
450: 5a (5a)
451: 05 (05)
452: 00 (00)
453: 87 (87)
454: 00 (00)
455: 1f (1f)
456: 00 (00)
457: 0f (0f)
458: 2f (2f)
459: 23 (23)
45a: 80 (80)
45b: 00 (00)
45c: 00 (00)
45d: e6 (e6)
CHK: e6 (e6) 

Lane status:
      LANE_DESKEW: 11111111
    BAD_DISPARITY: 00000000
     NOT_IN_TABLE: 00000000
 UNEXPECTED_KCHAR: 00000000
    CODE_GRP_SYNC: 11111111
       FRAME_SYNC: 11111111
    GOOD_CHECKSUM: 11111111
   INIT_LANE_SYNC: 11111111
FIFO_STATUS_REG_0: 00000000
FIFO_STATUS_REG_1: 00000000
fpga j_sync errs: 236500425

STPL test:
converter: 0, sample: 0, tp: 597a, fail: 0
converter: 0, sample: 1, tp: b2f3, fail: 0
converter: 0, sample: 2, tp: 0c6c, fail: 0
converter: 0, sample: 3, tp: 65e5, fail: 0
converter: 0, sample: 4, tp: bf5e, fail: 0
converter: 0, sample: 5, tp: 18d7, fail: 0
converter: 0, sample: 6, tp: 7250, fail: 0
converter: 0, sample: 7, tp: cbc9, fail: 0
converter: 0, sample: 8, tp: 2542, fail: 0
converter: 0, sample: 9, tp: 7e

0

# Write IQ sample waveforms
Check DAC output on a spectrum analyzer / scope.

In [ ]:
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

In [ ]:
# Triangle wave, seems distorted by the lower cut-off frequency
# of the DAC output balun transformer
# samples = arange(-2**15, 2**15, 512, dtype=int16)

# Positive and negative pulse
samples = zeros(4096, dtype=int16)
samples[100] = 32000
samples[140] = -32000

print(len(samples))
setSamples(r, samples)

# Main DDS (not used)

In [35]:
# Setup DDSes
wr(0x1E6, (1 << 1))             # Enable DDSM_EN_CAL_DC_INPUT (see Fig. 80) (tone on / off)
wr(0x112, (1 << 3) | (1 << 2))  # Enable NCO + Modulus
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

# setup main DDS frequency and amplitude
ad.setTone(1, 1e9, 1, f_ref=5.12e9)

DC_CAL_TONE: ff 50 
DDSM_FTW: 00 00 00 00 00 32 
DDSM_FTW_LOAD_ACK: 1


In [36]:
# DC test-mode off, enable JESD input
ad.ad.wr(0x1e6, 0)

0

In [53]:
# Adjust DDS phase offset
ad.setTone(1, phase=0)

DDSM_NCO_PHASE_OFFSET: 00 00 
